In [8]:
import QuantLib as ql
today = ql.Date(20, 6, 2025)
ql.Settings.instance().evaluationDate = today

calendar   = ql.NullCalendar()          # << every day is “good’’
dayCounter = ql.Actual365Fixed()

In [9]:
times   = [30/365, 90/365, 180/365, 365/365]      # years
strikes = [0.8, 0.9, 1.00, 1.10, 1.20]
vols = [[0.34, 0.31, 0.28, 0.26],
        [0.30, 0.28, 0.25, 0.23],
        [0.27, 0.25, 0.23, 0.22],
        [0.26, 0.24, 0.22, 0.21],
        [0.28, 0.26, 0.24, 0.23]]

volMatrix = ql.Matrix(len(strikes), len(times))
for i, row in enumerate(vols):
    for j, v in enumerate(row):
        volMatrix[i][j] = v

try:
    # constructor that takes times in *years* → fully calendar-free
    blackSurf = ql.BlackVarianceSurface(today, times, strikes,
                                        volMatrix, dayCounter)
except TypeError:
    # older bindings: fall back to dates but with NullCalendar
    expiries  = [today + int(t*365 + 0.5) for t in times]
    blackSurf = ql.BlackVarianceSurface(today, calendar, expiries,
                                        strikes, volMatrix, dayCounter)

blackSurf.enableExtrapolation()
volTS = ql.BlackVolTermStructureHandle(blackSurf)

In [14]:
S0, r, q = 1, 0.04, 0.0
spot     = ql.QuoteHandle(ql.SimpleQuote(S0))
ratesTS  = ql.YieldTermStructureHandle(ql.FlatForward(today, r, dayCounter))
divTS    = ql.YieldTermStructureHandle(ql.FlatForward(today, q, dayCounter))

process  = ql.BlackScholesMertonProcess(spot, divTS, ratesTS, volTS)

T = 180/365                             # 6-month put, K = 1.05
maturity = today + int(T*365 + 0.5)     # no holidays, just add days
payoff   = ql.PlainVanillaPayoff(ql.Option.Put, 1.05)
exercise = ql.AmericanExercise(today, maturity)
option   = ql.VanillaOption(payoff, exercise)

In [15]:
# works with QuantLib ≥ 1.30  (all params are positional)
engine = ql.FdBlackScholesVanillaEngine(
            process,        # Black-Scholes-MertonProcess
            200,            # tGrid      – time steps
            200,            # xGrid      – asset-price grid points
            0,              # dampingSteps
            ql.FdmSchemeDesc.CrankNicolson(),  # scheme
            True)           # localVol   – let the engine build Dupire σloc

option.setPricingEngine(engine)

print("NPV =", option.NPV())

NPV = 0.08673650237092757


In [1]:
import QuantLib as ql
today = ql.Settings.instance().evaluationDate

averageType = ql.Average.Arithmetic
option_type = ql.Option.Call

strike = 100.0
exerciseDate = ql.TARGET().advance(today, 90, ql.Days)

pastFixings = 0 # Empty because this is a new contract
asianFixingDates = [ql.TARGET().advance(today, x, ql.Days) for x in range(1,91)]

payoff = ql.PlainVanillaPayoff(option_type, strike)
exercise = ql.EuropeanExercise(exerciseDate)
option = ql.DiscreteAveragingAsianOption(averageType, 0.0, pastFixings, asianFixingDates, payoff, exercise)

initialValue = ql.QuoteHandle(ql.SimpleQuote(100))
sigma = 0.3
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual365Fixed()))
volTS = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), sigma, ql.Actual365Fixed()))
stochProcess = ql.BlackScholesProcess(initialValue, riskFreeTS, volTS)

engine = ql.FdBlackScholesAsianEngine(stochProcess, tGrid=100, xGrid=100, aGrid=50)

option.setPricingEngine(engine)
price = option.NPV()

print(f"Option price: {price}")

Option price: 4.3247473535576475
